In [ ]:
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles

using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_false_mPV200.0_dk_denom_15_kmax10.0_lmax30.0"
your_dir = dir*par

hbar = 1
c = 1
dk = π/15

NU = 300 # index i, labels rows, where mat[i,j] is a matrix
NV = 200 # index j, labels columns, where mat[i,j] is a matrix


In [ ]:
D = 3
Nr = 128*2^D + 3 # the overal course graining

out_dir = "./correlators/"*par*"/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");
(its_bln, all_filenames_bln)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="bilinears_");
(its_crlt, all_filenames_crlt) = list_h5_files(your_dir*"/data_$(Nr)", prefix="correlators_");

UU_crlt = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU).h5","crlt")
#VV_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt.h5","crlt")

println("length(its)=",length(its))
println("its",its)

println("length(its_bln)=",length(its_bln))
println("its_bln=",its_bln)

println("length(its_crlt)=",length(its_crlt))
println("its_crlt=",its_crlt)

In [ ]:
ti_max = length(its_bln)

U_tr = zeros(ti_max, length(r));
V_tr = zeros(ti_max, length(r));
f_tr = zeros(ti_max, length(r));
g_tr = zeros(ti_max, length(r));

Φ_tr = zeros(ti_max, length(r));
Π_tr = zeros(ti_max, length(r));
Ψ_tr = zeros(ti_max, length(r));

α_tr  = zeros(ti_max, length(r));
A_tr  = zeros(ti_max, length(r));
B_tr  = zeros(ti_max, length(r));
KB_tr = zeros(ti_max, length(r));

# classical
Ttt_tr = zeros(ti_max, length(r));
Ttr_tr = zeros(ti_max, length(r));
Trr_tr = zeros(ti_max, length(r));
#quantum <T_{ab}(t,r)>
Ttt_tr_q = zeros(ti_max, length(r));
Ttr_tr_q = zeros(ti_max, length(r));
Trr_tr_q = zeros(ti_max, length(r));

for i in 1:ti_max
    it = its[i]
    it_str  = lpad(it, 4, "0")
    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]

    U   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,17]
    V   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,18]
    f   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,15]
    g   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,16]

    Φ   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,1]
    Π   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,2]
    Ψ   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,3]

    α   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,11]
    A   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,4]
    B   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,5]
    KB  =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,9]

    bilin  = h5read(your_dir*"/data_$(Nr)/bilinears_$(it_str).h5","bilinears")

    U_tr[i,:] = U
    V_tr[i,:] = V
    f_tr[i,:] = f
    g_tr[i,:] = g

    Φ_tr[i,:] = Φ
    Π_tr[i,:] = Π
    Ψ_tr[i,:] = Ψ

    α_tr[i,:]  = α
    A_tr[i,:]  = A
    B_tr[i,:]  = B
    KB_tr[i,:] = KB

    # <Π^2> = Π^2 + (AB^2/α^2)*hbar*c^2*(bilin[2])/(4π)
    Π2_q = @. Π[:]^2 + hbar*c^2*real.(bilin[:,2])*A[:]*B[:]^2/(α[:]^2*4*π)
    # <Ψ^2> = Ψ^2 + hbar^c^2*bilin[3]/(4π)
    Ψ2_q = @. Ψ[:]^2 + hbar*c^2*real.(bilin[:,3])/(4*π)
    # <Π Ψ> = Π*Ψ + bilin[4]*hbar*c^2*B*A^(1/2)/(α*4*π)
    ΠΨ_q = @. Π[:]*Ψ[:] + real.(bilin[:,4])*hbar*c^2*B[:]*A[:]^0.5/(α[:]*4*π)
    # <(\partial_θ Φ)^2> = hbar*c^2*bilin[5]/(4*π)
    Φθ2_q = real.(bilin[:,5])*hbar*c^2/(4*π)

    # <T_tt> = α^2/(2A) [<Π^2>/B^2 + <Ψ^2>] + α^2/(B r^2) * <(\partial_θ Φ)^2>
    Ttt_tr_q[i,:] = @. α[:]^2/(2*A[:])*(Π2_q[:]/B[:]^2 + Ψ2_q[:]) + α[:]^2/(B[:]*r[:]^2)*Φθ2_q[:]
    # <T_tr> = α <Π Ψ> /(B A^0.5)
    Ttr_tr_q[i,:] = @. α[:]*ΠΨ_q[:]/(B[:]*A[:]^0.5)
    # <T_rr> = 0.5*[<Π^2>/B^2 + <Ψ^2>] - A/(B r^2) * <(\partial_θ Φ)^2>
    Trr_tr_q[i,:] = @. 0.5*(Π2_q[:]/B[:]^2 + Ψ2_q[:]) - A[:]/(B[:]*r[:]^2)*Φθ2_q[:]

    #classic
    # Π^2
    Π2 = @. Π[:]^2
    # Ψ^2 
    Ψ2 = @. Ψ[:]^2
    # Π*Ψ 
    ΠΨ = @. Π[:]*Ψ[:]
    # T_tt = α^2/(2A) [Π^2/B^2 + Ψ^2]
    Ttt_tr[i,:] = @. α[:]^2/(2*A[:])*(Π2[:]/B[:]^2 + Ψ2[:])
    # T_tr = α Π Ψ /(B A^0.5)
    Ttr_tr[i,:] = @. α[:]*ΠΨ[:]/(B[:]*A[:]^0.5)
    # T_rr = 0.5*[Π^2/B^2 + Ψ^2]
    Trr_tr[i,:] = @. 0.5*(Π2[:]/B[:]^2 + Ψ2[:])
    
end

dtdU_tr = 0.5./(α_tr.*f_tr);
drdU_tr = - 0.5./sqrt.(A_tr.*f_tr);

# T_UU = dt/dU*dt/dU*T_tt + 2*dt/dU*dr/dU*T_tr + dr/dU*dr/dU*T_rr

TUU_tr = @. Ttt_tr*dtdU_tr^2 + 2.0*Ttr_tr*dtdU_tr*drdU_tr + Trr_tr*drdU_tr^2;
TUU_tr_q = @. Ttt_tr_q*dtdU_tr^2 + 2.0*Ttr_tr_q*dtdU_tr*drdU_tr + Trr_tr_q*drdU_tr^2;

# T^VV = T_UU/g_UV^2
#g_UV = -1/(2*f*g)
gUV_tr = @. -1/(2*f_tr*g_tr);
TVVup_tr = @. TUU_tr/(gUV_tr^2);

dt = 64*dr/16
tlist = zeros(ti_max)
AH_t  = zeros(ti_max)
for i in 1:ti_max-1
    tlist[i+1] = tlist[i]+dt
    AH_t[i] = find_AH(r, B_tr[i,:], A_tr[i,:], KB_tr[i,:])
end
AH_t[end] = find_AH(r, B_tr[end,:], A_tr[end,:], KB_tr[end,:])
println(tlist)
#AH_t = AH_t +0.029296875*ones(length(tlist))
println(AH_t)

In [ ]:
#for i in 1:ti_max
#    for j in 1:Nr
#        if (A_tr.*f_tr)[i,j] <0
#            println("(i,j) = ", (i,j))
#        end
#    end
#end

In [ ]:
AH_tr = zeros(ti_max, length(r));
for i in 1:ti_max
    for j in 3:length(r)-1
        if (r[j]+r[j+1])/2 - AH_t[i] ≈ 0
            AH_tr[i,j] = 1
            AH_tr[i,j-1] = 1
            AH_tr[i,j-2] = 1
            AH_tr[i,j+1] = 1
            AH_tr[i,j+2] = 1
        end
    end
end

contourf(r[3:512], tlist, AH_tr[:,3:512],
color=:thermal,
wsize = (500,500),title = "AH_tr")

In [ ]:
p1 = contourf(r[3:100], tlist, U_tr[:,3:100],
color=:thermal,
wsize = (500,500),title = L"U(t,r)", ylabel = "t", xlabel="r")

p2 = contourf(r[3:100], tlist, V_tr[:,3:100],
color=:thermal,
wsize = (500,500),title = L"V(t,r)", ylabel = "t", xlabel="r")


plot(p1, p2, layout = (1, 2), legend = false, wsize = (1000,500) )


In [ ]:

#take min max of U(t,r), and V(t,r)
Umin = minimum(U_tr)
Umax = maximum(U_tr)
Vmin = minimum(V_tr)
Vmax = maximum(V_tr)

U_axis = zeros(NU)
for i in 1:NU
    U_axis[i] = Umin + (i-1)*(Umax - Umin)/(NU-1)
end

V_axis = zeros(NV)
for j in 1:NV
    V_axis[j] = Vmin + (j-1)*(Vmax - Vmin)/(NV-1)
end

dom      = NaN* zeros(NU,NV)
AH_UV    = NaN* zeros(NU,NV)
Φ_UV     = NaN* zeros(NU,NV)
TUU_UV   = NaN* zeros(NU,NV)
TUU_UV_q = NaN* zeros(NU,NV)
TVVup_UV = NaN* zeros(NU,NV)

Umin = minimum(U_tr)
Umax = maximum(U_tr)
Vmin = minimum(V_tr)
Vmax = maximum(V_tr)
println(Umin)
println(Umax)
println(Vmin)
println(Vmax)

dU = (Umax - Umin)/(NU-1)
#println(dU)
dV = (Vmax - Vmin)/(NV-1)
#println(dV)


In [ ]:
# new
for i in 1:ti_max#length(tlist)
    for j in 1:length(r)
        Utr     = U_tr[i,j]
        Vtr     = V_tr[i,j]
        Φtr     = Φ_tr[i,j]
        AHtr    = AH_tr[i,j]
        TUUtr   = TUU_tr[i,j]
        TUUtrq   = TUU_tr_q[i,j]
        TVVuptr = TVVup_tr[i,j]
        
        idown  = Int( round( 1 + (NU - 1)*(Utr - Umin)/(Umax - Umin) ) )
        jleft  = Int( round( 1 + (NV - 1)*(Vtr - Vmin)/(Vmax - Vmin) ) )

        if jleft < NV
            jright = jleft + 1
        else
            jright = jleft
        end
        if idown < NU
            iup =  idown + 1
        else
            iup =  idown
        end
        #println("iup=",iup)
        #println("idown=",idown)
        #println("jleft=",jleft)
        #println("jright=",jright)

        dom[idown, jleft]  = 1
        dom[iup, jleft]    = 1
        dom[idown, jright] = 1
        dom[iup, jright]   = 1
        
        Φ_UV[idown, jleft]  = Φtr
        Φ_UV[iup, jleft]    = Φtr
        Φ_UV[idown, jright] = Φtr
        Φ_UV[iup, jright]   = Φtr
   
        AH_UV[idown, jleft]  = AHtr
        AH_UV[iup, jleft]    = AHtr
        AH_UV[idown, jright] = AHtr
        AH_UV[iup, jright]   = AHtr
        
        TUU_UV[idown, jleft]  = TUUtr
        TUU_UV[iup, jleft]    = TUUtr
        TUU_UV[idown, jright] = TUUtr
        TUU_UV[iup, jright]   = TUUtr

        TUU_UV_q[idown, jleft]  = TUUtrq
        TUU_UV_q[iup, jleft]    = TUUtrq
        TUU_UV_q[idown, jright] = TUUtrq
        TUU_UV_q[iup, jright]   = TUUtrq

        TVVup_UV[idown, jleft]  = TVVuptr
        TVVup_UV[iup, jleft]    = TVVuptr
        TVVup_UV[idown, jright] = TVVuptr
        TVVup_UV[iup, jright]   = TVVuptr
    end
end


In [ ]:
contourf(V_axis, U_axis, dom[:,:],
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (500,500),title = "domain", frame=true)

#scatter!((AH_V, AH_U),label="AH(U,V)", mc=:red, ms=4, ma=0.1)


In [ ]:
contourf(V_axis, U_axis, AH_UV,
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (500,500),title = "domain", frame=true)


In [ ]:
AH_U = []#zeros(NU);
AH_V = []#zeros(NV);
for i in 1:NU
    for j in 1:NV
        if AH_UV[i,j]==1
            append!(AH_U, U_axis[i])
            append!(AH_V, V_axis[j])
            #AH_U[i] = U_axis[i]
            #AH_V[j] = V_axis[j]
        end
    end
end
println(AH_U)
println(AH_V)


In [ ]:
# scan the lists and place AH_V in increasing number; and follow the same flipts in AH_U
test=true
while test==true
    for i in 2:length(AH_V)
        if AH_V[i] < AH_V[i-1]
            temp1 = AH_V[i-1]
            AH_V[i-1] = AH_V[i]
            AH_V[i] = temp1
            temp2 = AH_U[i-1]
            AH_U[i-1] = AH_U[i]
            AH_U[i] = temp2
        end
    end
    j = 2
    while AH_V[j] >= AH_V[j-1] && j<length(AH_V)
        j+=1
        #println(j)
    end
    if j==length(AH_V)
        test = false
        #println(test)
    end
    #println("end of permutation")
end
    
println(AH_U)
println(AH_V)

In [ ]:
for i in 1:length(AH_V)
    println("i, V, U = ", (i, AH_V[i], AH_U[i]))
end

In [ ]:
#deleteat!(AH_U, (33,35,37,39))
#deleteat!(AH_V, (33,35,37,39))
#println(AH_U)
#println(AH_V)

In [ ]:
ui = 1
uj = 300
vi = 1
vj = 200
p = contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
        xlabel = "V",
        ylabel = "U",
        color=:thermal,
        wsize = (700,700),title = L"\Phi(U,V)",
        frame =true)
p=scatter!((AH_V, AH_U),label="AH(U,V)", markerstrokewidth=0, mc=:green, ms=3)
p =plot!([V_axis[100]], seriestype="vline", label="", linewidth=1.5, color = "black")

savefig(p, out_dir*"/Phi_UV.pdf")

In [ ]:
ui = 1
uj = 300
vi = 1
vj = 200
p = contourf(V_axis[vi:vj], U_axis[ui:uj], TUU_UV[ui:uj,vi:vj],
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),title = L"T_{UU}(U,V)",
    frame=true)
p = scatter!((AH_V, AH_U),label="AH(U,V)",  markerstrokewidth=0, mc=:green, ms=3)

savefig(p, out_dir*"/TUU_UV.pdf")

In [ ]:
ui = 1
uj = 300
vi = 1
vj = 200
p = contourf(V_axis[vi:vj], U_axis[ui:uj], TUU_UV_q[ui:uj,vi:vj],
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),title = L"\langle T_{UU}(U,V) \rangle",
    frame=true)
scatter!((AH_V, AH_U),label="AH(U,V)",  markerstrokewidth=0, mc=:green, ms=3)
#plot!([V_axis[100]], seriestype="vline", label="", linewidth=1.5, color = "black")

savefig(p, out_dir*"/TUU_UV_q.pdf")

In [ ]:
#println("AH_U = ",AH_U)
#println("AH_V = ",AH_V)
#ui  = 105
#i = 10
#j = 70
#contourf(V_axis[i:j], V_axis[i:j], real.(VV_crlt[ui,i:j,i:j]),
#    title="V-V correlator @ U = $(U_axis[ui])",
#    xlabel = "V",
#    ylabel = "V",
#    color=:thermal,
#    wsize = (500,500))

#plot!([AH_V[2]], seriestype="vline", label="AH", linewidth=3, color = "black")
#plot!([AH_V[2]], seriestype="hline", label="AH", linewidth=3, color = "black")

In [ ]:
println("AH_U = ",AH_U)
println("AH_V = ",AH_V)
vi  = 100
i = 160
j = 240

ah_vi = 69

contourf(U_axis[i:j], U_axis[i:j], real.(UU_crlt[vi,i:j,i:j]),
    title="U-U correlator @ V = $(V_axis[vi])",
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),# xlim=(-15,-2), ylim=(-15,-2),
    frame=true)

plot!([AH_U[ah_vi]], seriestype="vline", label="AH", linewidth=3, color = "black")
p = plot!([AH_U[ah_vi]], seriestype="hline", label="AH", linewidth=3, color = "black", legend=:topleft)

#scatter!((U_axis[i:j], U_axis[i:j]), markerstrokewidth=0, mc=:green, ms=5)

savefig(p, out_dir*"/UU_crlt_V_$(AH_V[ah_vi]).pdf")

In [ ]:
AH_V[68]